### Importing the Dependencies

In [123]:
import numpy as np
import pandas as pd
import os, sys
import yaml
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

### Data Collection and Analysis

PIMA Diabetes Dataset

In [154]:
# loading the diabetes dataset to a pandas DataFrame
diabetes_dataset = pd.read_csv('D:\Projects\Multiple Disease Prediction System\dataset\diabetes.csv')

In [125]:
# printing the first 5 rows of the dataset
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [126]:
# number of rows and Columns in this dataset
diabetes_dataset.shape

(768, 9)

In [127]:
diabetes_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [128]:
# getting the statistical measures of the data
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [129]:
diabetes_dataset['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

0 --> Non-Diabetic

1 --> Diabetic

In [130]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [131]:
# separating the data and labels
X = diabetes_dataset.drop(columns = 'Outcome', axis=1)
Y = diabetes_dataset['Outcome']

In [132]:
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [133]:
print(Y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


### Data transformation

In [134]:
def get_data_transformer_object():
            

    # define the steps for the preprocessor pipeline
    imputer_step = ('imputer', SimpleImputer(strategy='constant', fill_value=0))
    scaler_step = ('scaler', StandardScaler())

    preprocessor = Pipeline(
          steps=[
                imputer_step,
                scaler_step
                ]
            )
            
    return preprocessor

### Train Test Split

In [135]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [136]:
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


In [137]:
preprocessor = get_data_transformer_object()

In [138]:
X_train_scaled =  preprocessor.fit_transform(X_train)
X_test_scaled  =  preprocessor.transform(X_test)

In [139]:
directory = "D:/Projects/Multiple Disease Prediction System/saved transformation objects"
os.makedirs(directory, exist_ok=True)
preprocessor_path = os.path.join(directory, "diabetes_preprocessor.pkl")

In [140]:
with open(preprocessor_path, "wb") as file_obj:
       pickle.dump(preprocessor, file_obj)

### Training the Model

In [141]:
models = {
    'XGBClassifier': XGBClassifier(),
    'GradientBoostingClassifier' : GradientBoostingClassifier(),
    'SVC' : SVC(),
    'RandomForestClassifier': RandomForestClassifier(),
    'GaussianNB' : GaussianNB()
    }

In [142]:
def evaluate_models(X_train_scaled, X_test_scaled, Y_train, Y_test, models):

    report = {}

    for i in range(len(list(models))):
        model = list(models.values())[i]

        model.fit(X_train_scaled, Y_train)  # Train model

        y_train_pred = model.predict(X_train_scaled)

        y_test_pred = model.predict(X_test_scaled)

        train_model_score = accuracy_score(Y_train, y_train_pred)

        test_model_score = accuracy_score(Y_test, y_test_pred)

        report[list(models.keys())[i]] = test_model_score

    return report

In [143]:
def get_best_model(X_train_scaled, Y_train, X_test_scaled, Y_test):
            
             

    model_report: dict = evaluate_models(
        X_train_scaled=  X_train_scaled, 
        Y_train = Y_train, 
        X_test_scaled=  X_test_scaled, 
        Y_test = Y_test, 
        models = models
        )
        
        
    print(model_report)

    best_model_score = max(sorted(model_report.values()))

    ## To get best model name from dict

    best_model_name = list(model_report.keys())[
          list(model_report.values()).index(best_model_score)
          ]

    best_model_object = models[best_model_name]


    return best_model_name, best_model_object, best_model_score

In [144]:
def finetune_best_model(best_model_object: object, best_model_name, X_train_scaled, Y_train) -> object:

    # Use the standalone function read_yaml_file
    with open("D:/Projects/Multiple Disease Prediction System/config/model.yaml","rb") as yaml_file:
        model_param_grid = yaml.safe_load(yaml_file)
    
    model_param = model_param_grid["model_selection"]["model"][best_model_name]["search_param_grid"]

    grid_search = GridSearchCV(
        best_model_object, param_grid=model_param, cv=5, n_jobs=-1, verbose=1
    )
            
    grid_search.fit(X_train_scaled, Y_train)

    best_params = grid_search.best_params_

    print("Best params are:", best_params)

    finetuned_model = best_model_object.set_params(**best_params)

    return finetuned_model


In [145]:


filename = "D:/Projects/Multiple Disease Prediction System/config/model.yaml"

if not os.path.exists(filename):
    raise FileNotFoundError(f"The file '{filename}' does not exist.")


In [146]:
directory = "D:/Projects/Multiple Disease Prediction System/saved models"
os.makedirs(directory, exist_ok=True)
trained_model_path = os.path.join(directory, "diabetes_model.pkl")


In [147]:
def initiate_model_training(X_train_scaled, Y_train, X_test_scaled, Y_test):
            

    model_report: dict = evaluate_models(X_train_scaled =  X_train_scaled, Y_train = Y_train, X_test_scaled =  X_test_scaled, Y_test = Y_test, models = models)

    ## To get best model score from dict
    best_model_score = max(sorted(model_report.values()))

    ## To get best model name from dict

    best_model_name = list(model_report.keys())[
         list(model_report.values()).index(best_model_score)
         ]


    best_model = models[best_model_name]


    best_model = finetune_best_model(
         best_model_name= best_model_name,
         best_model_object= best_model,
         X_train_scaled = X_train_scaled,
         Y_train= Y_train
         )

    best_model.fit(X_train_scaled, Y_train)
    Y_pred = best_model.predict(X_test_scaled)
    best_model_score = accuracy_score(Y_test, Y_pred)
            
    print(f"best model name {best_model_name} and score: {best_model_score}")


    if best_model_score < 0.6:
        raise Exception("No best model found with an accuracy greater than the threshold 0.6")
    
    
    with open(trained_model_path, "wb") as file_obj:
       pickle.dump(best_model, file_obj)
    
    return

In [148]:
initiate_model_training(X_train_scaled, Y_train, X_test_scaled, Y_test)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best params are: {'var_smoothing': 0.001}
best model name GaussianNB and score: 0.7727272727272727
